## Primary DLT notebook containing all logic for bronze, silver, and gold table creation for Fitness Activity Lakehouse

In [0]:
import dlt
from pyspark.sql.functions import col, to_date, current_timestamp, when, sum as sum_, avg, count

### Bronze Layer - Ingest raw fitness activity data using Autoloader


In [0]:
@dlt.table(
  comment="Raw fitness activity ingested from landing zone using Autoloader."
)
def bronze_daily_activity():
    return (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.inferColumnTypes", "false")
        .schema("""
            Id STRING,
            ActivityDate STRING,
            TotalSteps INT,
            TotalDistance DOUBLE,
            TrackerDistance DOUBLE,
            LoggedActivitiesDistance DOUBLE,
            VeryActiveDistance DOUBLE,
            ModeratelyActiveDistance DOUBLE,
            LightActiveDistance DOUBLE,
            SedentaryActiveDistance DOUBLE,
            VeryActiveMinutes INT,
            FairlyActiveMinutes INT,
            LightlyActiveMinutes INT,
            SedentaryMinutes INT,
            Calories INT
        """)
        .load("s3://databricks-745bwkyiddeq9fthttjahg-cloud-storage-bucket/ohio-prod/3903799048317088/landing/bronze/daily_activity_stream/")
        .withColumn("ingestion_timestamp", current_timestamp())
    )

### Silver Layer - Clean and enrich Bronze data with data quality checks


In [0]:
@dlt.expect("valid_Id", "Id IS NOT NULL")
@dlt.expect("valid_ActivityDate", "ActivityDate IS NOT NULL")
@dlt.expect("Calories_positive", "Calories > 0")
@dlt.expect("TotalSteps_positive", "TotalSteps >= 0")
@dlt.table(
  comment="Cleaned fitness activity data with parsed dates and derived activity levels."
)
def silver_daily_activity_clean():
    df_bronze = dlt.read("bronze_daily_activity")
    return (df_bronze
        .withColumn("ActivityDate", to_date(col("ActivityDate"), "M/d/yyyy"))
        .withColumn("activity_level", 
                    when(col("TotalSteps") >= 10000, "Very Active")
                    .when((col("TotalSteps") >= 5000) & (col("TotalSteps") < 10000), "Moderately Active")
                    .otherwise("Lightly Active"))
        .dropna(subset=["Id", "ActivityDate", "TotalSteps", "Calories"])
    )

### Gold Layer - Daily fitness KPIs for dashboarding and ML training

In [0]:
# Gold Aggregation for Dashboarding
@dlt.table(
    name = "gold_daily_activity_dashboard",
    comment="Daily activity metrics with per-user values for dashboarding."
)
def create_gold_dashboard():
    df = dlt.read("silver_daily_activity_clean")
    return (
        df.groupBy("Id", "ActivityDate")
        .agg(
            sum_("TotalSteps").alias("total_steps"),
            sum_("Calories").alias("total_calories"),
            avg("TotalDistance").alias("avg_distance"),
        )
    )

In [0]:
# Gold Table Optimized for ML
@dlt.table(
    name = "gold_daily_activity_ml",
    comment="ML-ready daily activity data per user."
)
def create_gold_ml():
    df = dlt.read("silver_daily_activity_clean")
    return (
        df.groupBy("Id", "activity_level")
        .agg(
            avg("TotalSteps").alias("avg_steps"),
            avg("Calories").alias("avg_calories"),
            avg("TotalDistance").alias("avg_distance"),
            avg("TrackerDistance").alias("avg_tracker_distance"),
            avg("VeryActiveDistance").alias("avg_very_active_distance"),
            avg("ModeratelyActiveDistance").alias("avg_moderate_distance"),
            avg("LightActiveDistance").alias("avg_light_distance"),
            avg("SedentaryActiveDistance").alias("avg_sedentary_distance"),
            avg("VeryActiveMinutes").alias("avg_very_active_minutes"),
            avg("FairlyActiveMinutes").alias("avg_fairly_active_minutes"),
            avg("LightlyActiveMinutes").alias("avg_lightly_active_minutes"),
            avg("SedentaryMinutes").alias("avg_sedentary_minutes")
        )
    )